In [3]:
import soundfile as sf
import numpy as np
import scipy as signal
import matplotlib.pyplot as plt
from functools import lru_cache

In [43]:
# Load data
data1,samplerate = sf.read('/Users/hyh/Desktop/Test Files/Postfiltered/postfiltered_2020-01-08_09_25_03_1.raw', 
               channels=4, 
               samplerate=16000,
               subtype='PCM_16'
              )
data2,samplerate = sf.read('/Users/hyh/Desktop/Test Files/Postfiltered/postfiltered_2020-01-08_09_25_03_2.raw', 
               channels=4, 
               samplerate=16000,
               subtype='PCM_16'
              )

In [113]:
def truncate(sig1, sig2):
    '''
    This function truncates the longer signal and return two signals of the same length
    '''
    l1, l2 = len(sig1), len(sig2)
    if l1 <= l2:
        return sig1, sig2[0:l1]
    else:
        return sig1[0:l2], sig2

def prepadding(sig1, sig2, winDuration, Fs):
    l = winDuration * Fs
    N = len(sig1)    
    # Padding to have an interger number of windows
    if N%l != 0:
        padWidth = l-N%l
        sig1 = np.pad(sig1, (0, padWidth), 'constant', constant_values = 0)
        sig2 = np.pad(sig2, (0, padWidth), 'constant', constant_values = 0)
        
    numOfWindows = len(sig1)/l
    

def gcc_phat(sig1, sig2, fs=16000, max_tau=None, interp=1):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Generalized Cross Correlation - Phase Transform (GCC-PHAT)method.
    '''
    # make sure the length for the FFT is larger or equal than len(sig) + len(refsig)
    n = sig1.shape[0] 

    # Generalized Cross Correlation Phase Transform
    SIG1 = np.fft.rfft(sig1, n=n)
    SIG2 = np.fft.rfft(sig2, n=n)
    R = SIG1 * np.conj(SIG2)

    cc = np.fft.irfft(R / np.abs(R), n=(interp * n))

    max_shift = int(interp * n / 2)
    if max_tau:
        max_shift = np.minimum(int(interp * fs * max_tau), max_shift)

    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

    # find max cross correlation index
    shift = np.argmax(np.abs(cc)) - max_shift

    tau = np.float(shift / float(interp * fs))
    
    return tau, cc, R, SIG, SIG2

In [126]:
'''
Between two devices, each device generates 4 channels/arrays
The number of possible gcc-phat combinations is (8!)/(2!(8-2)!) = 28
'''
'''
         Fs: Sampling Frequency
WinDuration: The length of the moving window in time (s)
  tauPerWin: An array that keeps track of delay per window
'''

# This needs to be turned into a for loop for all 8 channels
# Need to make sure the number of windows are the same across all comparisons

sig1, sig2 = truncate(data1[:,0], data2[:,0])

winDuration, Fs = 1, 16000

    
for i in range(0,numOfWindows):
    sig1Temp, sig2Temp = sig1[i*l:l*(i+1)], sig2[i*l:l*(i+1)]
    
    




windowDuration, Fs = 1, 16000
window = np.hanning(16000)
max_tau = 5/340
# The number of samples in each window
l = windowDuration * Fs
N = len(sig1)
for i in range(98,99):
    sig1Temp, sig2Temp = sig1[i*l:l*(i+1)], sig2[i*l:l*(i+1)]
tau, cc, R, SIG, SIG2 = gcc_phat(sig1Temp*window, sig2Temp*window, fs=16000, max_tau=max_tau, interp=1)

In [4]:

# The number of samples in each window
l = windowDuration * Fs
N = len(sig1)
# Padding
if N%l != 0:
    padWidth = l-N%l
    sig1 = np.pad(sig1, (0, padWidth), 'constant', constant_values = 0)
    sig2 = np.pad(sig2, (0, padWidth), 'constant', constant_values = 0)
    
numOfWindows = len(sig1)/l
for i in range(100,101):
    sig1Temp, sig2Temp = sig1[i*l:l*(i+1)], sig2[i*l:l*(i+1)]
    [f,crossSpectrum] = signal.csd(sig1Temp, sig2Temp, 
                                   nperseg=l,                   
                                   fs=16000.0, 
                                   window='hamming',
                                   return_onesided=False, 
                                   scaling='spectrum')
    weight = 1/np.abs(crossSpectrum)
    Rxy = np.fft.ifft(np.multiply(weight,crossSpectrum))


C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in multiply


In [1]:
sig1Temp = (10**(-1))*np.ones(16000)

NameError: name 'np' is not defined

In [6]:
# @lru_cache
# def fact(n):
#     if n <= 1:
#         return 1
#     return n * fact(n-1)

# fact(8)/fact(2)/fact(6)

28.0